In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_extract
from functools import partial

In [2]:

spark = (
    SparkSession.builder.appName("ToParquetLeagueOfData")
    .master("spark://spark-master:7077")
    .config(
        "spark.jars.packages",
        "com.amazonaws:aws-java-sdk:1.11.563,com.amazonaws:aws-java-sdk-bundle:1.11.874,org.apache.hadoop:hadoop-aws:3.3.2,io.delta:delta-core_2.12:2.1.0,org.postgresql:postgresql:42.2.18"
    )
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.multipart.size", "838860800")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c13bfff3-7e7c-436a-a7bb-20834c53bad1;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.11.563 in central
	found com.amazonaws#aws-java-sdk-ioteventsdata;1.11.563 in central
	found com.amazonaws#aws-java-sdk-core;1.11.563 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.5 in central
	found org.apache.httpcomponents#httpcore;4.4.9 in central
	found commons-codec#commons-codec;1.10 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;

23/01/07 21:12:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#spark.stop()

In [4]:
spark.sparkContext.setLogLevel("ERROR")
sc = spark
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.access.key", "Vi6JWlVxIzuqHCXN")
hadoop_conf.set("fs.s3a.secret.key", "ZO17cOvijTE0CUGtEU80PscJHYkG8meD")
hadoop_conf.set("fs.s3a.path.style.access", "True")


In [47]:
matchs_df = spark.read.format("delta").load("s3a://league-of-data-gold/matchs")

matchs_game_creation_df = matchs_df.withColumn('gameCreation', F.from_unixtime(matchs_df.gameCreation / 1000 , "yyyy-MM-dd HH:mm:ss")) \
.withColumn('gameEndTimestamp', F.from_unixtime(matchs_df.gameEndTimestamp / 1000 , "yyyy-MM-dd HH:mm:ss")) \
.withColumn('gameStartTimestamp', F.from_unixtime(matchs_df.gameStartTimestamp / 1000 , "yyyy-MM-dd HH:mm:ss")) \
.withColumn('gameDuration', matchs_df.gameDuration / 60)

matchs_game_creation_df.write.mode("overwrite").format("jdbc")\
    .option("url", "jdbc:postgresql://data_warehouse:5432/data_lol_dw") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "match_detail") \
    .option("user", "data_lol").option("password", "data_lol").save()

summoner_df = spark.read.format("delta").load("s3a://league-of-data-gold/summoner/detail")

summoner_df.write.mode("overwrite").format("jdbc")\
    .option("url", "jdbc:postgresql://data_warehouse:5432/data_lol_dw") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "summoner_detail") \
    .option("user", "data_lol").option("password", "data_lol").save()